In [169]:
import os
import pandas as pd
from allennlp.predictors.predictor import Predictor
from gap_scorer import run_scorer  
from tqdm import tqdm
import torch


CUDA available, predictions will be faster.


error loading _jsonnet (this is expected on Windows), treating C:\Users\pc-bae-2\AppData\Local\Temp\tmpc0nkkxmr\config.json as plain json


In [ ]:

# Check for CUDA
if torch.cuda.is_available():
    print("CUDA available, predictions will be faster.")
else:
    print("CUDA not available, predictions may be slower.")
# Initialize the SpanBERT predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz",cuda_device=0)


In [171]:

# Read GAP data
main_path = "../gap-coreference-master/gap-coreference-master"  
main_path = "." 
gap_file =  "gap-test.tsv"
gap_development_path = os.path.join(main_path,gap_file)
gap_df = pd.read_csv(gap_development_path, delimiter='\t')
print(gap_df[:5])


       ID                                               Text Pronoun  \
0  test-1  Upon their acceptance into the Kontinental Hoc...     His   
1  test-2  Between the years 1979-1981, River won four lo...     him   
2  test-3  Though his emigration from the country has aff...      He   
3  test-4  At the trial, Pisciotta said: ``Those who have...     his   
4  test-5  It is about a pair of United States Navy shore...     his   

   Pronoun-offset             A  A-offset  A-coref                   B  \
0             383     Bob Suter       352    False              Dehner   
1             430        Alonso       353     True  Alfredo Di St*fano   
2             312  Ali Aladhadh       256     True              Saddam   
3             526       Alliata       377    False           Pisciotta   
4             406         Eddie       421     True         Rock Reilly   

   B-offset  B-coref                                             URL  
0       366     True      http://en.wikipedia.org/w

In [102]:
# Sample output from AllenNLP coref model
example = gap_df.iloc[2]
text = example['Text']
print(example)
result = predictor.predict(document=text)
print(result['clusters'])
# Initialize an empty list with placeholders
text_visual = ['_'] * len(result['document'])

# Fill in the placeholders with tokens
for i, token in enumerate(result['document']):
    text_visual[i] = token

# Add brackets for coref clusters
for cluster in result['clusters']:
    for start, end in cluster:
        text_visual[start] = '[' + text_visual[start]
        text_visual[end] = text_visual[end] + ']'

# Combine into a string
text_visual_str = ' '.join(text_visual)
print(text)
print(text_visual_str)


ID                                                    development-3
Text              He had been reelected to Congress, but resigne...
Pronoun                                                         his
Pronoun-offset                                                  265
A                                                           Angeloz
A-offset                                                        173
A-coref                                                       False
B                                                        De la Sota
B-offset                                                        246
B-coref                                                        True
URL               http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
Name: 2, dtype: object
[[[0, 0], [20, 22], [49, 51], [54, 54], [98, 101]], [[32, 32], [41, 43], [47, 47]], [[30, 30], [68, 68]], [[28, 28], [87, 87]], [[5, 5], [108, 108]]]
He had been reelected to Congress, but resigned in 1990 to accept a post as Amb

In [132]:
# Define the function to extract words that correspond to each cluster
def extract_cluster_words(tokenized_document, clusters):
    total_clusters = []
    for cluster in clusters:
        txt_cluster = []
        for start, end in cluster:
            cluster_tokens = tokenized_document[start:end+1]
            txt_cluster.append(" ".join(cluster_tokens))
        total_clusters.append(txt_cluster)
    return total_clusters

# Example 
tokenized_document = ['It', 'was', 'reported', 'that', 'John', 'and', 'Jane', 'were', 'together', '.', 'He', 'said', 'it', 'was', 'true', '.']
clusters = [[[4, 4], [10, 10]], [[5, 5], [12, 12]]]
print(result['clusters'])
print(extract_cluster_words(result['document'], result['clusters']))


[['Simon', 'Simon', 'he', "Simon 's", "Simon 's"],
 ["Cheryl Cassidy , Pauline 's friend and also a year 11 pupil in Simon 's class",
  'her',
  'her',
  'her'],
 ['her boyfriend', 'he', 'him'],
 ['Dumped', 'this'],
 ["Pauline 's", 'her friend Pauline']]

In [174]:
# Create batches - not working ngl
batch_size = 16 
num_batches = len(gap_df) // batch_size + (1 if len(gap_df) % batch_size != 0 else 0)

predictions = []
num_rows = len(gap_df)
num_batches = (num_rows + batch_size - 1) // batch_size
for batch_idx in tqdm(range(num_batches), desc="Processing batches"):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, num_rows)
        batch = gap_df.iloc[start_idx:end_idx]
        for _, row in  batch.iterrows():
            # print(row)
            text = row['Text']
            result = predictor.predict(document=text)
            clusters = result['clusters']
            tokens = result['document']

            # Initialize coreference indicators for A and B to 0
            a_coref, b_coref = 0, 0

            # Find the cluster containing the pronoun
            pronoun_offset = row['Pronoun-offset']
            pronoun_length = len(row['Pronoun'])
            pronoun_cluster = None

            # Calculate the character offsets for each token
            char_offsets = []
            offset = 0
            token_idx = 0
            
            tokens_dict = {}
            for i, char in enumerate(text):
                if char == ' ':
                    continue
                if text[i:i+len(tokens[token_idx])] == tokens[token_idx]:
                    start_offset = i
                    end_offset = i + len(tokens[token_idx]) - 1
                    char_offsets.append((start_offset, end_offset))
                    tokens_dict[ (start_offset, end_offset)] =tokens[token_idx]

                    i = end_offset
                    token_idx += 1
                    if token_idx >= len(tokens):
                        break
                    
            # print(tokens_dict)
            # print('clusters',clusters)
            # clusters = sorted(clusters, key=len) #Sort base on how cluster size, smaller is more important!
            
            # print('sorted clusters',clusters)
            pronoun_clusters = []
            for cluster in clusters:
                for start, end in cluster:
                    # print(char_offsets[start])
                    try:
                        start_offset, end_offset = char_offsets[start]
                        if start_offset <= pronoun_offset and end_offset >= (pronoun_offset + pronoun_length - 1):
                            pronoun_cluster = cluster
                            # print("Pronoun cluster:",cluster)
                            pronoun_clusters.append(pronoun_cluster)
                        # break
                    except Exception as e:
                        print("Warning - Error with selecting clusters, gonna ignore it and continue but be ware")
                        print(e)
                        continue
                # if pronoun_cluster:
                #     break
            # print(extract_cluster_words(tokens,[pronoun_cluster]))
            # Check if 'A' or 'B' is in the same cluster as the pronoun
            for pronoun_cluster in pronoun_clusters:
                a_start, a_end = row['A-offset'], row['A-offset'] + len(row['A']) - 1
                b_start, b_end = row['B-offset'], row['B-offset'] + len(row['B']) - 1
                # print("A goal:",a_start,a_end)
                # print("B goal:",b_start,b_end)
                for start, end in pronoun_cluster:
                    # print("start,end",start,end)
                    start_offset, _ = char_offsets[start]
                    _, end_offset =char_offsets[end]
                    # print("start,end char:", start_offset, end_offset)
                    if start_offset <= a_start and end_offset >= a_end and b_coref == 0:
                        a_coref = 1
                        break
                    if start_offset <= b_start and end_offset >= b_end and a_coref == 0:
                        b_coref = 1
                        break

            predictions.append({
                'ID': row['ID'],
                'A-coref': a_coref,
                'B-coref': b_coref
            })
            # print(predictions)
            # break


Processing batches: 100%|██████████| 125/125 [02:18<00:00,  1.11s/it]


In [ ]:
import pandas as pd

# Convert the predictions dictionary to a DataFrame
predictions_df = pd.DataFrame(predictions)
# Convert 1 to True and 0 to False in the A-coref and B-coref columns
predictions_df['A-coref'] = predictions_df['A-coref'].astype(bool)
predictions_df['B-coref'] = predictions_df['B-coref'].astype(bool)
# Save the DataFrame to a TSV file
predictions_df.to_csv('predictions-test.tsv', sep='\t', index=False)
# Gold_annotations is a list of ground truth annotations from GAP
# Run the scorer
scores = run_scorer('gap-development.tsv', 'predictions-test.tsv')

print(scores)


## Debugging stuff

In [115]:
clu = []
tks = result['document']
total_clusters = []
for cluster in result['clusters']:
    print('New Cluster')
    
    txt_cluster = []
    for start, end in cluster:
        print(start,end)
        print(tks[start:end+1])
        txt_cluster.append(" ".join(tks[start:end+1]) )
    total_clusters.append(txt_cluster)
print(total_clusters)

    

New Cluster
0 0
['He']
20 22
['De', 'la', 'Sota']
49 51
['De', 'la', 'Sota']
54 54
['his']
98 101
['De', 'la', 'Sota', "'s"]
New Cluster
32 32
['Defeated']
41 43
['this', 'latter', 'setback']
47 47
['it']
New Cluster
30 30
['1991']
68 68
['1991']
New Cluster
28 28
['C*rdoba']
87 87
['C*rdoba']
New Cluster
5 5
['Congress']
108 108
['Congress']
[['He', 'De la Sota', 'De la Sota', 'his', "De la Sota 's"], ['Defeated', 'this latter setback', 'it'], ['1991', '1991'], ['C*rdoba', 'C*rdoba'], ['Congress', 'Congress']]


In [85]:
# Initialize variables
char_offsets = []
offset = 0
token_idx = 0

# Iterate over the original text to calculate character-based offsets
for i, char in enumerate(text):
    # Skip spaces
    if char == ' ':
        continue
    
    # Check if the current character matches the start of the next token
    if text[i:i+len(result['document'][token_idx])] == result['document'][token_idx]:
        start_offset = i
        end_offset = i + len(result['document'][token_idx]) - 1
        char_offsets.append((start_offset, end_offset))
        
        # Move the pointer i to the end of the current token
        i = end_offset
        
        # Move to the next token
        token_idx += 1
        
        # Exit the loop if we've found all tokens
        if token_idx >= len(result['document']):
            break

# Debugging: Print each token next to its offset
count = 0
for (start, end), token in zip(char_offsets, result['document']):
    print(f"Token: {token}, Start Offset: {start}, End Offset: {end}, Count {count}")
    count +=1
    


Token: He, Start Offset: 0, End Offset: 1, Count 0
Token: had, Start Offset: 3, End Offset: 5, Count 1
Token: been, Start Offset: 7, End Offset: 10, Count 2
Token: reelected, Start Offset: 12, End Offset: 20, Count 3
Token: to, Start Offset: 22, End Offset: 23, Count 4
Token: Congress, Start Offset: 25, End Offset: 32, Count 5
Token: ,, Start Offset: 33, End Offset: 33, Count 6
Token: but, Start Offset: 35, End Offset: 37, Count 7
Token: resigned, Start Offset: 39, End Offset: 46, Count 8
Token: in, Start Offset: 48, End Offset: 49, Count 9
Token: 1990, Start Offset: 51, End Offset: 54, Count 10
Token: to, Start Offset: 56, End Offset: 57, Count 11
Token: accept, Start Offset: 59, End Offset: 64, Count 12
Token: a, Start Offset: 66, End Offset: 66, Count 13
Token: post, Start Offset: 68, End Offset: 71, Count 14
Token: as, Start Offset: 73, End Offset: 74, Count 15
Token: Ambassador, Start Offset: 76, End Offset: 85, Count 16
Token: to, Start Offset: 87, End Offset: 88, Count 17
Token:

In [156]:
gold_annotations = []

for index, row in gap_df.iterrows():
    gold_annotation = {
        'ID': row['ID'],
        'A-coref': True if row['A-coref'] == True else False,
        'B-coref': True if row['B-coref'] == True else False,
        # Add other fields as needed
    }
    gold_annotations.append(gold_annotation)
gold_annotations[:5]

[{'ID': 'development-1', 'A-coref': True, 'B-coref': False},
 {'ID': 'development-2', 'A-coref': True, 'B-coref': False},
 {'ID': 'development-3', 'A-coref': False, 'B-coref': True},
 {'ID': 'development-4', 'A-coref': False, 'B-coref': True},
 {'ID': 'development-5', 'A-coref': False, 'B-coref': True}]